# Set Up New Environment

You will likely need to train for >24 hours.  Colab will disconnect you.  You must be prepared to restart training when this eventually happens.  Training is divided into ticks, every so many ticks (50 by default) your neural network is evaluated and a snapshot is saved.  When CoLab shuts down, all training after the last snapshot is lost. It might seem desirable to snapshot after each tick; however, this snapshotting process itself takes nearly an hour.  It is important to learn an optimal snapshot size for your resolution and training data.

We will mount GDRIVE so that your snapshots are saved there.  You must also place your training images in GDRIVE.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


You must also install NVIDIA StyleGAN2 ADA PyTorch.  We also need to downgrade PyTorch to a version that supports StyleGAN.

In [3]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.6 kB/s 
     |████████████████████████████████| 17.4 MB 808 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python

# Convert Your Images

In [ ]:
/content/drive/My\ Drive/Colab Notebooks/Health/

In [4]:
!python /content/drive/My\ Drive/Colab\ Notebooks/Health/stylegan2-pytorch/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/My\ Drive/Colab\ Notebooks/Health/DATASET/benign_256 --dest /content/drive/My\ Drive/Colab\ Notebooks/Health/torch_data_benign_256

100% 437/437 [00:06<00:00, 65.08it/s] 


The following command can be used to clear out the newly created dataset.  If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [ ]:
#!rm -R /content/drive/MyDrive/data/gan/dataset/circuit/*

# Perform Initial Training

In [5]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/My\ Drive/Colab\ Notebooks/Health/torch_res_b256"
DATA = "/content/drive/My\ Drive/Colab\ Notebooks/Health/torch_data_benign_256"
SNAP = 10

In [ ]:
# Build the command and run it
cmd = f"python3 /content/drive/My\ Drive/Colab\ Notebooks/Health/stylegan2-pytorch/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/My Drive/Colab Notebooks/Health/torch_data_benign_256",
    "use_labels": false,
    "max_size": 437,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "